In [1]:
import pandas as pd
import numpy as np

from itertools import combinations_with_replacement as combrep
from itertools import permutations

import seaborn as sns

# Vorlesung: Kopierbeziehungen von Diagrammen

## Kopierbeziehungen

Drei mögliche Szenarien treten für die Kopierbeziehungen zwischen zwei Diagrammen A und B auf. A kann eine Kopie von B sein, oder B eine Kopie von A, wenn beide Diagramme eine direkte Verwandschaftsbeziehung haben. Natürlich könnten auch beide Diagramme von einer gemeinsamen Vorlage C kopiert worden sein, die z.B. verloren gegangen ist.

Zum Festlegen einer Kopierbeziehung nutzen wir die Merkmale der Diagramme. Bei einem Kopiervorgang will ein Kopist alle Merkmale einer Vorlage übernehmen.  Durch Fehler können zufällig Merkmale verloren gehen. Es ist aber eher unwahrscheinlich, dass ein Kopist Merkmale hinzufügt, die in einer Vorlage nicht vorhanden waren. 

Diese Kopierbeziehungen können durch eine einfache Anweisung kodiert werden. Wir vergleichen alle Stellen eines Beschreibungsvektors für ein Diagramm A mit einem anderen Diagramm B. Ist in A ein Merkmal vorhanden, aber in B nicht (1->0), weisen wir diesem Kopiervorgang eine Wahrscheinlichkeit von P zu. Das heißt es ist wahrscheinlich zufällige Fehler zu machen. Ist in A hingegen ein Merkmal nicht vorhanden, in B aber schon (0->1), ordnen wir diesem Vorgang die Wahrscheinlichkeit P(1-P) zu, wie oben argumentiert. Auch der wenn sich in A und B jeweils das selbe Merkmal finden (1->1), ordnen wir eine Wahrscheinlichkeit zu (1-P), da nicht immer ideale Kopien gemacht werden. 

In [2]:
def kopierbeziehungen(p,u,v):                # p ist die Wahrscheinlichkeit für einen fehlerfreien Kopierprozess, u und v sind Merkmalsvektoren
    prop = []
    for i in range(len(u)):                  # für jedes Paar von Merkmalseinträgen überprüfen wir die Kopierbeziehung
        if u[i] > v[i]:                      # Wahrscheinlichkeit für irgendeinen Fehler ist P = (1-p) 
            prop.append((1-p))
        elif u[i] < v[i]:                    # Wahrsch. zufällig ein bestimmtes Merkmal hinzuzufügen ist P=p(1-p)^(Anzahl der Merkmale)
            prop.append(p*(1-p)**(len(u)-1))
        elif u[i] == v[i]:                   # Wahrsch. eine Kopie ohne Fehler herzustellen ist P=p
            prop.append(p)
        else:
            pass
    res = np.product(prop)                   # Die gesamte Wahrscheinlichkeit für eine Kopierbeziehung von u -> v ist das Produkt aller Teil-Wahrscheinlichkeiten 
    return print('{} -> {}: p='.format(u,v),res)

Mit der Funktion zip(u,v) können wir die Merkmalsvektoren zweier Diagramme verknüpfen. Gleiche Stellen des Vektors werden zu einem Tupel, einer nicht veränderbaren Gruppe, zusammengefasst. Für jedem Eintrag des Tupels prüfen wir, ob der Eintrag von A größer, kleiner oder gleich dem Eintrag von B ist. Aus der so entstanden Vergleichsliste für alle Stellen des Merkmalsvektors bilden wir dann das Produkt, welches die Gesamtwahrscheinlichkeit der Kopierbeziehung beschreibt. 

Für zwei hypothetische Diagramme A und B haben wir folgende Merkmale verzeichnet.

In [3]:
A = [1,1,1]
B = [0,1,1]
C = [1,1,0]

Nun testen wir, welche Kopierfolge wahrscheinlicher ist.

In [4]:
kopierbeziehungen(p=1/30,u=A,v=B)

[1, 1, 1] -> [0, 1, 1]: p= 0.00107407407407


In [5]:
kopierbeziehungen(p=1/30,u=B,v=A)

[0, 1, 1] -> [1, 1, 1]: p= 3.46090534979e-05


Wie zuvor festgelegt, ist ein Verlust zweier Merkmale deutlich wahrscheinlicher, als das Hinzufügen. Daher ist B wahrscheinlich eine Kopie von A. 

Das Testen der dritten Hypothese ist mit den zwei Merkmalsvektoren nicht möglich. Hier für müssen wir Informationen über alle möglichen anderen Vektoren haben. Dies erfolgt im nächsten Abschnitt.

## Testen aller möglichen Kopierbeziehungen

Um die Möglichkeit einer gemeinsamen Kopiervorlage zu überprüfen, konstruieren wir als erstes alle `2^N` = 8 möglichen Ausprägungen des Merkmalsvektors. Bei drei Merkmalen gibt es je eine Konfiguration mit allen bzw. keinen vorhandenden Merkmalen. Ein oder zwei Merkmale können auf je 3 verschiedene Arten auftreten. Diese Möglichkeiten sind durch die Binomialverteilung beschrieben (Länge des Vektors über Anzahl vorhandener Merkmale).

In [6]:
l = [list(set(permutations(x))) for x in combrep(range(2),3)]
ll = [[x] for y in l for x in y]
ll

[[(0, 0, 0)],
 [(1, 0, 0)],
 [(0, 1, 0)],
 [(0, 0, 1)],
 [(0, 1, 1)],
 [(1, 1, 0)],
 [(1, 0, 1)],
 [(1, 1, 1)]]

Für die Erzeugung des Dataframes übergeben wir die Liste mit dem gewünschtem Spaltennamen.

In [7]:
dfSimulation = pd.DataFrame(ll,columns=['vek'])
dfSimulation['vek'] = dfSimulation['vek'].apply(lambda r: list(r))
dfSimulation.head(2)

,vek
0,"[0, 0, 0]"
1,"[1, 0, 0]"


In [8]:
def kopierbez(p, u, v):
    res = np.product([(1-p) if i[0] > i[1] else (p)*(1-p)**(len(u)-1) if i[0] < i[1] else (p) for i in zip(u,v)])
    return res

Um die Kopierbeziehung zweier Vektoren zu erhalten, wenden wir die oben definierte Funktion auf die Spalte 'vek' an. Für jeden möglichen Vektor j fügen wir eine neue Spalte mit Namen 'j' zu dem Dataframe hinzu, deren Elemente `a_ij` die Wahrscheinlichkeit beschreiben, dass j eine Kopie von i ist. 

In [9]:
for x in dfSimulation.index:
    dfSimulation['{}: {}'.format(x,dfSimulation['vek'].iloc[x])] = dfSimulation['vek'].apply(lambda r: kopierbez(1/3,r, dfSimulation['vek'].iloc[x]))
dfSimulation.head(2)

,vek,"0: [0, 0, 0]","1: [1, 0, 0]","2: [0, 1, 0]","3: [0, 0, 1]","4: [0, 1, 1]","5: [1, 1, 0]","6: [1, 0, 1]","7: [1, 1, 1]"
0,"[0, 0, 0]",0.037037,0.016461,0.016461,0.016461,0.007316,0.007316,0.007316,0.003252
1,"[1, 0, 0]",0.074074,0.037037,0.032922,0.032922,0.014632,0.016461,0.016461,0.007316


Wir setzen nun den Merkmalsvektor als neuen Index und wenden einen Filter auf die Einträge an. Wahrscheinlichkeit die kleiner als 0.05 sind, werden 0 gesetzt. 

In [10]:
dfSimulation = dfSimulation.set_index('vek')

In [11]:
dfSimulation = dfSimulation.applymap(lambda x: x if (x > 0.015) else 0)
dfSimulation.head(2)

,"0: [0, 0, 0]","1: [1, 0, 0]","2: [0, 1, 0]","3: [0, 0, 1]","4: [0, 1, 1]","5: [1, 1, 0]","6: [1, 0, 1]","7: [1, 1, 1]"
vek,,,,,,,,
"[0, 0, 0]",0.037037,0.016461,0.016461,0.016461,0.0,0.000000,0.000000,0.0
"[1, 0, 0]",0.074074,0.037037,0.032922,0.032922,0.0,0.016461,0.016461,0.0


Der Dataframe dfSimulation beschreibt nun alle möglichen Kopierbeziehungen. 

## Untersuchung zu Epiphänomenen

Um mögliche gemeinsame Kopiervorlagen zu suchen, konzentrieren wir uns auf die zwei Merkmalsvektoren (1,1,0) und (1,0,1). Um die hohen Wahrscheinlichkeiten besser sichtbar zu machen, unterlegen wir die Zellen in Abhängigkeit ihres Wertes farblich. 

Da wir mögliche Vorlagen suche, von denen zwei Vektoren Kopien sind, müssen wir die Spalten der "Ziel"-Merkmalsvektoren auswählen. 

In [12]:
cm = sns.light_palette((210, 90, 60), input="husl", as_cmap=True)

s = dfSimulation[['5: [1, 1, 0]','6: [1, 0, 1]']].reset_index().style.background_gradient(cmap=cm)
s

Die Wahrscheinlichkeit, dass (1,1,0) eine Kopie von (1,0,1) ist, oder auch andersrum, ist gering. 

Wir suchen einen Merkmalsvektor, von dem beide Ziel-Vektoren mit genügend hoher Wahrscheinlichkeit Kopien sein können. Die bei weitem höchste Wahrscheinlichkeit führt hier auf den Vektor (1,1,1).

Die Vermutung wäre daher, dass die beiden durch die Merkmalsvektoren beschriebenen Diagramme nicht direkt voneinander kopiert wurden, sondern von einer gemeinsamen Vorlage.